<a href="https://colab.research.google.com/github/erha500/IMLO-Open-Assessment/blob/main/IMLO_Open_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms as T
from torchvision.transforms import v2

In [16]:
train_transform_augment = v2.Compose([
    v2.Resize([224,224]),
    #v2.RandomPerspective(),
    v2.RandomHorizontalFlip(0.5),
    v2.RandomRotation(45),
    #v2.RandomVerticalFlip(0.5)
    #v2.RandomAutocontrast(),
    #v2.GaussianBlur(kernel_size=3),
    v2.ToTensor(),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_transform = v2.Compose([
  v2.Resize([224,224]),
  v2.ToTensor(),
  v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


test_transform = v2.Compose([
  v2.Resize([224,224]),
  v2.ToTensor(),
  v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Downloading Flowers102 dataset from datasets

In [17]:
training_data_augment = datasets.Flowers102(root="dataset", split="train", transform=train_transform_augment, download=True)

training_data = datasets.Flowers102(root="dataset", split="train", transform=train_transform, download=True)

val_data = datasets.Flowers102(root="dataset", split="val", transform=test_transform, download=True)

test_data = datasets.Flowers102(root="dataset", split="test", transform=test_transform, download=True)

In [18]:
batch_size = 64

#Create data loaders
train_dataloader = DataLoader(training_data_augment, batch_size = batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 3, 224, 224])
Shape of y: torch.Size([64]) torch.int64


In [19]:
#Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.model = nn.Sequential(
      #Convolutional layers
      nn.Conv2d(3, 64, kernel_size=3, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(2, stride=2),
      nn.Conv2d(64,64, kernel_size=3, padding=1),
      nn.BatchNorm2d(64),
      nn.MaxPool2d(2, stride=2),

      nn.ReLU(),
      nn.Conv2d(64, 128, kernel_size=3, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.MaxPool2d(2, stride=2),
      nn.Conv2d(128,128, kernel_size=3, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.MaxPool2d(2, stride=2),

      nn.Conv2d(128, 256, kernel_size=3, padding=1),
      nn.BatchNorm2d(256),
      nn.ReLU(),
      nn.MaxPool2d(2, stride=2),
      nn.Conv2d(256,256, kernel_size=3, padding=1),
      nn.BatchNorm2d(256),
      nn.ReLU(),
      nn.MaxPool2d(2, stride=2),

      #Linear layers
      nn.Flatten(),

      nn.Linear(256 * 3 * 3, 1024),
      nn.BatchNorm1d(1024),
      nn.ReLU(),
      nn.Dropout(0.15),

      nn.Linear(1024, 512),
      nn.BatchNorm1d(512),
      nn.ReLU(),
      nn.Dropout(0.15),

      nn.Linear(512,256),
      nn.BatchNorm1d(256),
      nn.ReLU(),
      nn.Dropout(0.15),

      nn.Linear(256,102),
      nn.BatchNorm1d(102),
      nn.ReLU(),

      nn.LogSoftmax(dim=1)
    )

  def forward(self, x):
    return self.model(x)

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_

In [20]:
#loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.NLLLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.005) #weight decay is L2 regularisation
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=5, factor=0.5)

In [21]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    #Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 5 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [22]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct) :>0.1f}%, Avg loss: {test_loss:>8f} \n")

    accuracy = 100 * correct
    return accuracy, test_loss

In [23]:
epochs = 100

best_accuracy = 0
patience = 10
triggers = 0

for t in range(epochs):
  print(f"Epoch {t+1} \n ---------------------------------------")

  """#train on original data
  train_dataloader = DataLoader(training_data, batch_size=batch_size)
  train(train_dataloader, model, loss_fn, optimizer)"""

  #train on augmented data
  train(train_dataloader, model, loss_fn, optimizer)

  #testing on evaluation data
  accuracy, test_loss = test(val_dataloader, model, loss_fn)

  scheduler.step(test_loss)

  if accuracy > best_accuracy:
    best_accuracy = accuracy
    triggers = 0
    torch.save(model.state_dict(), "model.pt")
    print("Saved model at current state")
  else:
    triggers += 1

  if triggers > patience:
    print("Stopping early")
    break

print("Testing -----------------------")
model.load_state_dict(torch.load("model.pt"))
model.eval()
test(test_dataloader, model, loss_fn)
print("Done!!!!")

Epoch 1 
 ---------------------------------------
loss: 4.902709 [   64/ 1020]
loss: 4.734595 [  384/ 1020]
loss: 4.501976 [  704/ 1020]
loss: 4.454731 [  960/ 1020]
Test Error: 
 Accuracy: 1.0%, Avg loss: 4.634351 

Saved model at current state
Epoch 2 
 ---------------------------------------
loss: 4.577503 [   64/ 1020]
loss: 4.185880 [  384/ 1020]
loss: 4.384799 [  704/ 1020]
loss: 4.356721 [  960/ 1020]
Test Error: 
 Accuracy: 4.4%, Avg loss: 4.482550 

Saved model at current state
Epoch 3 
 ---------------------------------------
loss: 4.167769 [   64/ 1020]
loss: 4.101056 [  384/ 1020]
loss: 3.907490 [  704/ 1020]
loss: 3.796858 [  960/ 1020]
Test Error: 
 Accuracy: 12.8%, Avg loss: 4.109963 

Saved model at current state
Epoch 4 
 ---------------------------------------
loss: 4.072965 [   64/ 1020]
loss: 3.788932 [  384/ 1020]
loss: 3.938277 [  704/ 1020]
loss: 3.835911 [  960/ 1020]
Test Error: 
 Accuracy: 16.8%, Avg loss: 3.891199 

Saved model at current state
Epoch 5 
 ----